<a href="https://colab.research.google.com/github/Onmang/Multimedia_engineering/blob/master/t20me022_0714.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.10.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 76.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0

In [2]:
import tensorflow
tensorflow.__version__

'2.10.0'

In [3]:
# 形態素解析モジュールのインストール
!pip install mecab-python3
!pip install unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.6/581.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=c733f9310f4e7c17b6b1a0d9de8244c16f8be806cf0733108adf5d5988d41fef
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic-lite


In [4]:
# 必要なモジュールのインポート
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, Masking, Input, Embedding, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences # パディング用
import numpy as np
import MeCab

In [5]:
wakati = MeCab.Tagger('-Owakati')  #Mecab。分かち書きオプションを利用
# 青空文庫から「吾輩は猫である」を利用
# wagahai.txtは西崎がテキスト整形したものであることに注意
with open('/content/sample_data/wagahai.txt', 'r') as f:
    sents = [l.strip() for l in f.readlines()]
num_sents = len(sents) # 文の数を取得しておく

# プレーンテキストを形態素解析して，一旦ファイルに保存しつつ（wagahai-wakati.txt），単語のリストを作成する
vocab = set() # vocab（語彙）集合の初期化
with open('wagahai-wakati.txt', 'w', encoding='utf-8') as f:
    for s in sents:
        line = wakati.parse(s).strip().split() # 形態素解析を実行
        vocab.update(line) # 語彙集合を更新
        f.write( ' '.join(line) ) # ファイルに書き込み
        f.write('\n') # 行ごとに改行する
vocab = sorted(vocab) # ソートして集合をリスト化する
vocab.insert(0, 'EOS') # 文末記号（End of Sentence）を追加。IDは0
vocab.insert(1, 'BOS') # 文頭記号（Begine of Sentence）を追加。IDは1
print(vocab)
# vocab.index('吾輩') # デバッグ：単語のIDを調べる
vocab_size = len(vocab) + 1 # 未知語分を1つ加算（単語リストにない単語の場合のID割当て）
vocab = [w.encode('utf-8') for w in vocab] # バイナリに変換しておく

['EOS', 'BOS', '&', 'A', 'Do', 'Quid', 'a', 'aliud', 'amicitiae', 'boy', 'c', 'est', 'h', 'inimica', 'iomelesidonophrunicherata', 'mulier', 'nisi', 'r', 'see', 'the', 'you', '—', '…', '※', '○', '、', '。', '々', '「', '」', '〔', '〕', 'あ', 'ああ', 'ああら', 'あい', 'あいつ', 'あいにく', 'あえか', 'あえて', 'あかんべえ', 'あがる', 'あき', 'あきらめ', 'あきらめる', 'あきれ', 'あきれ返っ', 'あく', 'あくび', 'あくまで', 'あくる日', 'あけ', 'あける', 'あげ', 'あご', 'あさま', 'あした', 'あしらっ', 'あす', 'あすこ', 'あずかり', 'あたかも', 'あたっ', 'あたり', 'あたり前', 'あちら', 'あっ', 'あつかっ', 'あて', 'あと', 'あながち', 'あなた', 'あの', 'あははは', 'あばか', 'あばた', 'あぶない', 'あまく', 'あまり', 'あやしい', 'あやまる', 'あやまれ', 'あら', 'あららか', 'あらわせ', 'あらわれ', 'あり', 'ありがたい', 'ありがとう', 'ありさま', 'ありゃ', 'ある', 'あるい', 'あるいは', 'あるき', 'あるく', 'あるけ', 'あれ', 'あろう', 'あんな', 'あんまり', 'い', 'いい', 'いいえ', 'いいかけ', 'いいや', 'いう', 'いえ', 'いか', 'いかな', 'いかなる', 'いかに', 'いかめしい', 'いかん', 'いき', 'いきなり', 'いく', 'いくら', 'いけ', 'いけす', 'いける', 'いささか', 'いざ', 'いざっ', 'いじっ', 'いじめ', 'いずれ', 'いたずら', 'いただい', 'いただか', 'いただき', 'いただきゃ', 'いっ', 'いっこう', 'いっしょ', 'いっそ', 'いつ', 'いつ頃', 'いで', 'いな', 'い

In [6]:
# テキストをtf.dataを使ってロード（tf.Tensor型で読み込む）
line_ds = tf.data.TextLineDataset('wagahai-wakati.txt')
tf.print(next(iter(line_ds))) # 最初の１文の表示

吾輩 は 猫 で ある 。 名前 は まだ 無い 。


In [7]:
# ある1文の単語列をID列に変換する
def word2id(sent):
    unkid = len(vocab) # vocabにない単語があった場合のID（未知語）
    idseq = []
    for word in sent.split():
        if word in vocab:
            idseq.append(vocab.index(word)) # 単語のIDを保存
        else: # 単語が語彙集合に登録されていなかったら（今回に限ってはこれは起きない）
            idseq.apend(unkid) # 未知語のID
    #idseq = [vocab.index(word) for word in sent.split()]
    #idseq.insert(0, 1) # 先頭に"BOS"を入れる
    return idseq

# ミニバッチデータの作成
def make_padding_data(sents):
    batch_data = [] # データを入れるリスト
    label_data = [] # データに対するラベルを入れるリスト
    ### 1文ずつ処理
    for s in sents:
        #print('DEBUG', s)
        train = word2id( s.numpy() ) # ID列に変換
        label = train.copy() # 教師ラベル
        train.insert(0, 1) # 先頭に"BOS"（ID=1）を入れる
        label.append(0) # 教師ラベルの文末に"EOS"（ID=0）を入れる
        #print('train', len(train))
        #print('label', len(label))
        batch_data.append(train) # 訓練の1文データをリストに追加
        label_data.append(label) # ラベルも追加
    # バッチデータの長さを揃えるため，　短いデータの後ろに0を埋める。
    batch_data = pad_sequences(batch_data, padding='post', value=0, dtype=np.int32)
    label_data = pad_sequences(label_data, padding='post', value=0, dtype=np.int32)
    return batch_data, label_data

# テキストをtf.dataを使ってロード（tf.Tensor型で読み込む）
batch_size = 10
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = tf.data.TextLineDataset('wagahai-wakati.txt')
train_ds = train_ds.shuffle(buffer_size=num_sents) # 完全シャッフル
train_ds = train_ds.repeat(1) # 繰り返し
# 今回はミニバッチサイズを固定にしないといけないため，drop_remainderオプションで設定したバッチサイズに満たない訓練データを捨てる設定にする
train_ds = train_ds.batch(batch_size, drop_remainder=True) # ミニバッチ生成

# ミニバッチ単位で，インデックスから訓練データに変換
train_ds = train_ds.map(lambda x: tf.py_function(make_padding_data, [x], Tout=[tf.int32, tf.int32]))
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE) # プレフェッチの設定（実行速度アップのため）

In [8]:
"""
モデル定義
"""
# 入力データの長さ（時間）は未定で，各時刻のデータは1次元（ワンホットベクトル）
input = Input(shape=(None,), name='input') # バッチサイズを指定する必要あり
# マスク層。全次元が0のデータを無視するようにするための層
masked = Masking(mask_value=0, name='mask')(input)
embed = Embedding(vocab_size, 100, name='embed')(masked) # 単語のエンベッド次元を100とする
h = LSTM(128, name='lstm1', return_sequences=True)(embed)
# 各時刻の出力をDense層にいれないといけないので，TimeDistributedラッパーを使う。
h = TimeDistributed( tf.keras.layers.Dense(128, activation='relu', name='fc1'))(h)
h = Dropout(0.5)(h) # ドロップアウト
output = TimeDistributed( tf.keras.layers.Dense(vocab_size, activation='softmax', name='output'))(h)
model = tf.keras.Model(inputs=input, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, None)]            0         
                                                                 
 mask (Masking)              (None, None)              0         
                                                                 
 embed (Embedding)           (None, None, 100)         547700    
                                                                 
 lstm1 (LSTM)                (None, None, 128)         117248    
                                                                 
 time_distributed (TimeDistr  (None, None, 128)        16512     
 ibuted)                                                         
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                             

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_ds, epochs=200, use_multiprocessing=True) # まともな作文をさせるには500-1000エポック回したほうがよい（時間かかる…）

Epoch 1/200
225/225 [==============================] - 13s 26ms/step - loss: 3.4445 - sparse_categorical_accuracy: 0.6045
Epoch 2/200
225/225 [==============================] - 5s 24ms/step - loss: 2.3697 - sparse_categorical_accuracy: 0.6362
Epoch 3/200
225/225 [==============================] - 5s 24ms/step - loss: 2.2754 - sparse_categorical_accuracy: 0.6560
Epoch 4/200
225/225 [==============================] - 7s 30ms/step - loss: 2.1830 - sparse_categorical_accuracy: 0.6645
Epoch 5/200
225/225 [==============================] - 6s 28ms/step - loss: 2.1489 - sparse_categorical_accuracy: 0.6665
Epoch 6/200
225/225 [==============================] - 7s 33ms/step - loss: 2.0755 - sparse_categorical_accuracy: 0.6782
Epoch 7/200
225/225 [==============================] - 6s 25ms/step - loss: 1.9801 - sparse_categorical_accuracy: 0.6910
Epoch 8/200
225/225 [==============================] - 8s 35ms/step - loss: 1.9109 - sparse_categorical_accuracy: 0.6973
Epoch 9/200
225/225 [==========

KeyboardInterrupt: ignored

In [11]:
# （時間がないので…）訓練済みモデルをロードして評価してみる
tmodel = tf.keras.models.load_model('/content/sample_data/wagahai_lm_700.h5')
# 同じモデルになっているか確認する
tmodel.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, None)]            0         
                                                                 
 mask (Masking)              (None, None)              0         
                                                                 
 embed (Embedding)           (None, None, 100)         547700    
                                                                 
 lstm1 (LSTM)                (None, None, 128)         117248    
                                                                 
 time_distributed (TimeDistr  (None, None, 128)        16512     
 ibuted)                                                         
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                             

In [12]:
# 作文用関数
def make_sentence(firstword):
    tmodel.reset_states() # 最初にLSTMの状態をリセットする
    sentence = [firstword]
    wid = vocab.index('BOS'.encode('utf-8')) # 最初の文頭記号
    wid = np.array([wid])
    wid = wid[np.newaxis, :]
    pred = tmodel.predict(wid).argmax() # 文頭記号をモデルに入れる
    wid = vocab.index(firstword.encode('utf-8')) # 最初の単語をID化
    for i in range(10):  # 最長10単語
        wid = np.array([wid])
        wid = wid[np.newaxis, :]
        pred = tmodel.predict(wid).argmax()
        #print(vocab[pred].decode()) # 予測単語を表示
        sentence.append(vocab[pred].decode()) # 予測した単語をリストに追加
        wid = pred
        if wid == 0: # EOSが来たら終了
            break
    print(f'作文結果：{" ".join(sentence)}')

make_sentence('吾輩')

1/1 [==============================] - 0s 19ms/step
作文結果：吾輩 と 云い ます — — — — — — —
